# Explicações

[playlist com explicações do trabalho](https://www.youtube.com/playlist?list=PLcvOyD_LMr6mH7Z_2fBoihUdprWn8vMwC)



# Versão Simplificada de uma GPU

* Warps com 4 threads
* Banco de Registradores com 64 registros
* Máximo de 32 threads por Multiprocessador
* Máximo de 4 blocos por Multiprocessador
* Máximo de 16 threads por bloco
* Instruções
  * Ld 4 ciclos cache, 20 ciclos memoria
  * Sd 4 ciclos cache, 20 ciclos memoria
  * Add e Mul 3 ciclos
* Ordem de swap dos Warps é aleatória   

## Exemplo 1

* 2 warps
* Codigo a = b + c


```
ld r1; b
ld r2; c
add r1,r1,r2; b+c
sd r1; a = b+c
```

Supor que os dados estão na cache.


In [ ]:
#@title Trecho codigo a = b + c
%%writefile exemplo.csv
PC; Inst; Warp; Fetch; Decode; Exec; W
0; Ld r1; w1; 1; 2; 3-6; 7;
1; Ld r2; w1; 2; 3; 4-7; 8;
0; Ld r1; w2; 3; 4; 5-8; 9;
1; Ld r2; w2; 4; 5; 6-9; 10;
x; nop; ; 5
x; nop; ; 6
2; add r1,r1,r2; w1; 7; 8; 9-11; 12
x; nop; ; 8
2; add r1,r1,r2; w1; 9; 10; 11-13; 14
x; nop; ; 10
3; sd r1; 11; 12; 13-17
x; nop; ; 12
3; sd r1; 13; 14; 15-19

Writing exemplo.csv


In [ ]:
#@title Imprime
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# create checkbox widget
checkbox = widgets.Checkbox(
    value=False,
    description='Edit table',
    indent=False
)

# Lê o arquivo CSV e carrega em um dataframe
df = pd.read_csv('exemplo.csv', delimiter=';',index_col=False)
df = df.fillna("-")
from google.colab import data_table
# define function to display dataframe
def show_df(use_display):
    clear_output()  # clear current output
    if use_display:
        display(df)
    else:
        print(df)
    display(checkbox)


#data_table.DataTable(data.airports(), include_index=False, num_rows_per_page=10)

# display checkbox and dataframe
display(checkbox)
show_df(checkbox.value)

# define callback for checkbox widget
def on_checkbox_change(change):
    show_df(change.new)

# attach callback to checkbox widget
checkbox.observe(on_checkbox_change, names='value')


,PC,Inst,Warp,Fetch,Decode,Exec,W
0,0,Ld r1,w1,1,2,3-6,7.0
1,1,Ld r2,w1,2,3,4-7,8.0
2,0,Ld r1,w2,3,4,5-8,9.0
3,1,Ld r2,w2,4,5,6-9,10.0
4,x,nop,,5,-,-,-
5,x,nop,,6,-,-,-
6,2,"add r1,r1,r2",w1,7,8,9-11,12.0
7,x,nop,,8,-,-,-
8,2,"add r1,r1,r2",w1,9,10,11-13,14.0
9,x,nop,,10,-,-,-


Checkbox(value=True, description='Edit table', indent=False)

Este exemplo usa 2 Warps com 4 threads. Cada thread precisa de 2 registradores, então usa no total $2W \cdot 4th \cdot 2$= 16 registradores dos 64 registradores da GPU.


In [ ]:
#@title Uso do Fetch pelos Warps
%%writefile exemplo.csv
warp; 1;2;3;4;5;6;7;8;9;0;1;2;3
w1; ld; ld; ; ; ; ; add; ; ; ; sd
w2;   ;   ; ld; ld; ; ; ; ; add; ; ; ; sd

Overwriting exemplo.csv


In [ ]:
#@title Imprime
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# create checkbox widget
checkbox = widgets.Checkbox(
    value=False,
    description='Edit table',
    indent=False
)

# Lê o arquivo CSV e carrega em um dataframe
df = pd.read_csv('exemplo.csv', delimiter=';',index_col=False)
df = df.fillna("-")
from google.colab import data_table
# define function to display dataframe
def show_df(use_display):
    clear_output()  # clear current output
    if use_display:
        display(df)
    else:
        print(df)
    display(checkbox)


#data_table.DataTable(data.airports(), include_index=False, num_rows_per_page=10)

# display checkbox and dataframe
display(checkbox)
show_df(checkbox.value)

# define callback for checkbox widget
def on_checkbox_change(change):
    show_df(change.new)

# attach callback to checkbox widget
checkbox.observe(on_checkbox_change, names='value')


,warp,1,2,3,4,5,6,7,8,9,0,1,2.1,3.1
0,w1,ld,ld,,,,,add,,,,sd,-,-
1,w2,,,ld,ld,,,,,add,,,,sd


Checkbox(value=True, description='Edit table', indent=False)

# Parte I

1. Elaborar um exemplo com mais Warps sem violar o máximo de registradores que mantenha a GPU sempre ocupada.

2. **(2 pontos extras)** Fazer um código que leia a planilha de execução e gera o diagrama de tempo do Warp. Não precisa ser com saída "planilha" e tem que ter uma opção de barra de "scroll" na horizontal caso o diagrama seja grande. Usar cores diferentes para Warps diferentes.

# Parte II

1. Elaborar um exemplo com mais Warps sem violar o máximo de registradores, blocos e outras restrições que mantenha a GPU sempre ocupada, supondo agora que os dados não estão na cache.


# Parte III

1. Elaborar dois exemplos com intensidade aritmética 5 e 10, respectivamente. Sem violar o máximo de registradores, blocos e outras restrições que mantenha a GPU sempre ocupada, supondo agora que os dados não estão na cache.


# Parte IV (**2 pontos extras**)

Elaborar  4 exemplos que estejam na "borda" para  violar (no limite) as restrições (um para cada restrição) abaixo e precisem de pelo menos 3 multiprocessadores para executar simultaneamente.

1. Banco de Registradores com 64 registros
2. Máximo de 32 threads por Multiprocessador
3. Máximo de 4 blocos por Multiprocessador
4. Máximo de 16 threads por bloco

# Questao 4 Prova 3

A GPU turing tem as seguintes características

* blocos de até 1024 threads
* 64KB shared por SMX
* 64K registradores (256KB) por SMX
* 2048 threads concorrentes por SMX
* Maximo  de 32 blocos



## Exemplos de enunciados para Prova

1) Quantos blocos irão executar concorrentemente no multiprocessador se cada thread usa 32 registradores, o bloco tem 512 threads, bloco usa 32Kbyte de memoria shared.

Resp: 2 blocos devido ao limite de Shared.

2) Quantos blocos irão executar concorrentemente no multiprocessador se cada thread usa 32 registradores, o bloco tem 1024 threads, bloco usa 16Kbyte de memoria shared.

resp:
 * 2 blocos devido ao máximo de 2 blocos * 1024 threads * 32 regs = 64K regs
 * 2 blocos * 1024 threads = 2048 máximo de threads

3) Quantos blocos irão executar concorrentemente no multiprocessador se cada thread usa 64 registradores, o bloco tem 1024 threads, bloco usa 16Kbyte de memoria shared.

* 1 bloco = 64 reg * 1024 threads = 64K registradores

4) Quantos blocos irão executar concorrentemente no multiprocessador se cada thread usa 8 registradores, o bloco tem 256 threads, bloco usa 2Kbyte de memoria shared.

resp: 8 blocos devido ao limite de 2048 threads por multiprocessador.





